In [1]:
from transformers import pipeline, AutoTokenizer, GPTNeoXForCausalLM, GPTNeoXConfig
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from tqdm import tqdm
import plotly.express as px
import pandas as pd
import torch
import os

tqdm.pandas()

/home/mchorse/.local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class HFMemoriesDataset(Dataset):
    is_dataframe = False

    def __init__(self, memories, tokenizer, sample=None):
        self.tokenizer = tokenizer
        self.memories = memories
        if sample is not None:
            self.memories = self.memories.to_pandas().sample(sample)
            self.is_dataframe = True

    def __getitem__(self, index):
        memory_record = (
            self.memories.iloc[index] if self.is_dataframe else self.memories[index]
        )
        decoded_text = self.tokenizer.decode(memory_record["tokens"])
        return decoded_text

    def __len__(self):
        return len(self.memories)


def load_tokenizer(split_name):
    isDeduped = split_name.startswith("deduped")
    model = split_name.split("duped.")[-1]
    corresponding_model = f"EleutherAI/pythia-{model}{'-deduped' if isDeduped else ''}"
    tokenizer =  AutoTokenizer.from_pretrained(corresponding_model)
    tokenizer.pad_token = tokenizer.eos_token
    return tokenizer

def load_model(split_name):
    isDeduped = split_name.startswith("deduped")
    model = split_name.split("duped.")[-1]
    corresponding_model = f"EleutherAI/pythia-{model}{'-deduped' if isDeduped else ''}"
    device = torch.device("cuda:7")
    return GPTNeoXForCausalLM.from_pretrained(corresponding_model, device_map="auto", offload_folder="offload").eval()

In [3]:
split_name = "deduped.12b"
memories = load_dataset("EleutherAI/pythia-memorized-evals")[split_name]
tokenizer = load_tokenizer(split_name)
memories_dataset = HFMemoriesDataset(
    load_dataset("EleutherAI/pythia-memorized-evals")[split_name], 
    tokenizer)

pythia_model = load_model(split_name)

Found cached dataset parquet (/home/mchorse/.cache/huggingface/datasets/EleutherAI___parquet/EleutherAI--pythia-memorized-evals-623aaa371a33821a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 16/16 [00:00<00:00, 31.29it/s]
Found cached dataset parquet (/home/mchorse/.cache/huggingface/datasets/EleutherAI___parquet/EleutherAI--pythia-memorized-evals-623aaa371a33821a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 16/16 [00:00<00:00, 39.55it/s]


In [51]:
def calculate_perplexity(logits, labels):
    # Store the probabilities for each token. These will be summed later, but having the 
    # individual probabilities is helpful for debugging.
    token_probs = []

    # Don't include the final token logits. There are no labels for these since the sequence has ended.
    shifted_logits = logits.detach()[:-1, :]

    for token_index in range(len(shifted_logits)):
        # Map the logits to probabilities.
        predicted_probs = torch.softmax(shifted_logits[token_index], dim=0)
        # Get the probability of the correct label.
        label_prob = predicted_probs[labels[token_index + 1]]
        # Store the probability for this token.
        token_probs.append(label_prob.detach())
    
    # Caluclate the log-likelyhood of the sequence by summing the probabilities of each token and then taking the log.
    log_likelihood = torch.log(torch.stack(token_probs)).sum()
    
    # Caluclate the cross entropy by dividing the negative log-likelihood by the number of tokens.
    cross_entropy = -log_likelihood / len(shifted_logits)

    # Calculate the perplexity by taking the exponential of the cross entropy.
    perplexity = torch.exp(cross_entropy).item()
    return perplexity


data_loader = DataLoader(memories_dataset, batch_size=32)
hf_perplexities = []
all_perplexities = []

with torch.no_grad():
    for batch in [memories_dataset[0]]:
        tokenized_batch = tokenizer(
            batch, return_tensors="pt", max_length=512, truncation=True, padding=True
        )
        tokenized_batch.to(torch.device("cuda:7"))
        labels = tokenized_batch["input_ids"]

        outputs = pythia_model(**tokenized_batch, labels=labels)
        logits = outputs.logits.detach()
        hf_perplexities += [torch.exp(outputs.loss).item()]

        all_perplexities += [calculate_perplexity(logits[i], labels[i]) for i in range(len(logits))]

print(hf_perplexities)
print(all_perplexities)

[4.78515625]
[4.78515625]


In [ ]:
sequence = memories_dataset[0]
display(sequence)

tokenized_batch = tokenizer(sequence, return_tensors="pt", max_length=512, truncation=True, padding=True)
tokenized_batch["attention_mask"][0][1:] = 0
tokenized_batch.to(torch.device("cuda:7"))
labels = tokenized_batch["input_ids"]

likelihoods = None
with torch.no_grad():
    for i in range(2, len(labels[0])):
        # tokenized_batch["attention_mask"][0][:i] = 1
        outputs = pythia_model(**tokenized_batch, labels=labels)
        probs = torch.softmax(outputs.logits, dim=0)
        # likelyhood = outputs.logits[0][labels[0][i]]
        loss = outputs.loss * i
        likelihoods = loss if likelihoods is None else likelihoods + outputs.loss

log_likelihoods = torch.log(likelihoods)
perplexity = torch.exp(-log_likelihoods / len(labels[0])).item()
perplexity



In [ ]:
memories_df = memories.to_pandas()[["index"]]
memories_df["perplexity"] = all_perplexities
memories_df[["index", "perplexity"]].to_csv(f"./datasets/memories_{split_name}.csv", index=False)
memories_df

In [ ]:
with torch.no_grad():
    test_sequence = memories_dataset[0]
    perplexity = pipeline("text-generation", model=pythia_model, tokenizer=tokenizer, device=0, metric="perplexity")
    perplexity_score = perplexity(test_sequence)[0]["score"]
    perplexity_score